<a href="https://colab.research.google.com/github/Chigaga/Dimimmings/blob/master/%22processing_dimmings%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install chart-studio

     |████████████████████████████████| 71kB 3.5MB/s 


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

from IPython.display import HTML

py.sign_in("chigaga", "zPIazOZh5yTY1Uqn9z71")



In [4]:
tables_folder = Path('/content/drive/My Drive/Phd project/tables_folder')
sectors = pd.read_csv(tables_folder / '20111001.csv')
sectors

,Unnamed: 0,Unnamed: 1,area,brightness,interval,lat,lon,sector
0,166,0,5.831654e+06,-15.746248,0.666667,67.934821,-1.167568,3.0
1,166,1,5.827212e+06,-18.039982,0.666667,67.935499,-0.697511,3.0
2,166,2,5.822883e+06,-15.929602,0.666667,67.934658,-0.227295,3.0
3,166,3,5.826001e+06,-15.690197,0.666667,68.119173,-1.187040,3.0
4,166,4,5.821534e+06,-17.668350,0.666667,68.119908,-0.713227,3.0
...,...,...,...,...,...,...,...,...
4585231,599,13500,5.592571e+06,-49.504417,4.616667,87.924363,52.945970,5.0
4585232,599,13501,5.586769e+06,-66.715240,4.616667,87.779318,55.664713,5.0
4585233,599,13502,5.581051e+06,-57.630424,4.616667,87.629845,58.047199,5.0
4585234,599,13503,5.593432e+06,-0.000000,4.616667,88.022665,57.071914,5.0


In [15]:
#@title forward-backward exp smoothing
def exp_smooth_forward(data, alpha, init=0):
    "Supplementary function for forward exponential smoothing"
    out = np.empty(data.size)
    out[0] = init
    for i in range(1, data.size):
        out[i] = out[i-1] + alpha*(data[i]-out[i-1])
    return out

def exp_smooth_backward(data, alpha, init=0):
    "Supplementary function for backward exponential smoothing"
    out = np.empty(data.size)
    out[-1] = init
    for i in reversed(range(data.size-1)):
        out[i] = out[i+1] + alpha*(data[i]-out[i+1])
    return out
    
def exp_smooth(data, alpha, init=0):
    "Supplementary function for forward then backward exponential smoothing"
    return exp_smooth_backward(exp_smooth_forward(data, alpha, init), alpha, exp_smooth_forward(data, alpha, init)[-1])

In [68]:
#@title function for plotting sector parameters in time
#function to make line plots for 5 sectors
generate_report = False
def plot_traces(sector_list, var, xlable = 'xlable', ylable = 'ylable',
                legend = legend_list, title = 'title', mode='lines' ):

    #plot_races.counter += 1
    #fig_name = 'images/' + str(plot.counter) + '.jpg'
    fig = go.Figure()
    i=0
    for sec in sector_list:
        fig.add_trace(go.Scatter(x=sec['time'], y=sec[var], mode=mode, name = 'sector '+legend_list[i], showlegend = True))
        i+=1
    fig.update_layout(
        title=go.layout.Title(
            text=title,
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=xlable
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=ylable
            )
        )
    )
       
    if generate_report is True:
        fig.write_image(fig_name)
        display(Image(fig_name))
    else:
        fig.show()
#plot_traces.counter = 0

In [42]:
#@title function time derivative
def time_deriv (sec,param):
    y = sec[param]
    x = sec['time']
    dy = np.zeros(sec[param].shape,np.float)
    dy[0:len(y)-1] = np.diff(y)/np.diff(x)/3600
    dy[-1] = dy[-2]
    return dy

In [59]:
legend_list = ['1','2','3','4','5']

## Preparing data for plotting

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Creating new columns in the frame

In [5]:
##we drop unuseful columns
sectors.rename({"Unnamed: 0":"a","Unnamed: 1":"b" }, axis="columns", inplace=True)
sectors.drop(["a","b"], axis=1, inplace=True)

#time ffom the first dimming appearance
sectors['time'] = sectors['interval']-sectors['interval'].min()
sectors['time'] = np.around(sectors['time'],decimals=3)
sectors['lon_class'] = np.around(sectors['lon'],decimals=-1)
sectors['number'] = 1 #to count pixels for mean brightness

#adding distance information
deg_dist = 12.17 #distance of 1 degree in Mm
sectors['ring'] = (90-sectors["lat"]).apply(np.ceil) 
sectors['ring'] = sectors['ring']-sectors['ring'].min()+1
sectors['dist'] = sectors['ring']*deg_dist

#splitting the frame for sectors back again - double number not to mix with distance plot
sec11 = sectors.query('sector == 1.0')
sec22 = sectors.query('sector == 2.0')
sec33 = sectors.query('sector == 3.0')
sec44 = sectors.query('sector == 4.0')
sec55 = sectors.query('sector == 5.0')

sectors

,area,brightness,interval,lat,lon,sector,time,lon_class,number,ring,dist
0,5.831654e+06,-15.746248,0.666667,67.934821,-1.167568,3.0,0.00,-0.0,1,23.0,279.91
1,5.827212e+06,-18.039982,0.666667,67.935499,-0.697511,3.0,0.00,-0.0,1,23.0,279.91
2,5.822883e+06,-15.929602,0.666667,67.934658,-0.227295,3.0,0.00,-0.0,1,23.0,279.91
3,5.826001e+06,-15.690197,0.666667,68.119173,-1.187040,3.0,0.00,-0.0,1,22.0,267.74
4,5.821534e+06,-17.668350,0.666667,68.119908,-0.713227,3.0,0.00,-0.0,1,22.0,267.74
...,...,...,...,...,...,...,...,...,...,...,...
4585231,5.592571e+06,-49.504417,4.616667,87.924363,52.945970,5.0,3.95,50.0,1,3.0,36.51
4585232,5.586769e+06,-66.715240,4.616667,87.779318,55.664713,5.0,3.95,60.0,1,3.0,36.51
4585233,5.581051e+06,-57.630424,4.616667,87.629845,58.047199,5.0,3.95,60.0,1,3.0,36.51
4585234,5.593432e+06,-0.000000,4.616667,88.022665,57.071914,5.0,3.95,60.0,1,2.0,24.34


preparing a frame grouped by sectors for plotting

In [6]:
#summing all values for finding brightness and area  (ee)
sum_by_sec = sectors.groupby(['time','sector']).sum().reset_index()
sum_by_sec['mean br'] = sum_by_sec['brightness']/sum_by_sec['number']
sum_by_lon = sectors.groupby(['time','lon_class']).sum().reset_index()

sum_by_sec_dist = sectors.groupby(['time','sector','ring','dist']).size().to_frame('count').reset_index()
#sum_by_lon

,time,lon_class,area,brightness,interval,lat,lon,sector,number,ring,dist
0,0.000,-10.0,1.851824e+08,-466.574914,21.333333,2326.916199,-187.999810,96.0,32,568.0,6912.56
1,0.000,-0.0,1.244995e+09,-3284.196785,144.000000,15371.579718,-203.602912,720.0,216,4175.0,50809.75
2,0.007,-20.0,2.962620e+08,-856.598976,34.340142,3997.772785,-899.216948,153.0,51,618.0,7521.06
3,0.007,-10.0,5.996943e+08,-1578.464412,70.026956,7915.481766,-922.427692,312.0,104,1495.0,18194.15
4,0.007,-0.0,1.579987e+09,-4084.291986,184.494094,19472.251126,-283.135908,916.0,274,5325.0,64805.25
...,...,...,...,...,...,...,...,...,...,...,...
6069,3.950,20.0,3.554524e+09,-9678.554472,2945.433333,50359.471270,12048.919771,2552.0,638,7379.0,89802.43
6070,3.950,30.0,1.528391e+09,-5951.186196,1274.200000,22902.422097,8138.354514,1216.0,276,2075.0,25252.75
6071,3.950,40.0,9.288132e+08,-3272.285898,775.600000,14167.752523,6630.031061,840.0,168,1035.0,12595.95
6072,3.950,50.0,6.028141e+08,-2224.675718,503.216667,9295.657025,5336.084928,545.0,109,570.0,6936.90


## Animation of area/brightness depending on the polar angle

In [7]:
plot_br = sum_by_lon.copy()
plot_br['brightness']=plot_br['brightness']/10000
fig2 = px.area(plot_br, x='lon_class', y='brightness',
                  title='<b>Intensity decrease depending on the polar angle</b>', animation_frame="time",)#, line_close=True)
fig2.update_layout(
    xaxis_title="Longitude [deg]",
    yaxis_title='Brightness [10^4 DN]')
fig2.update_layout(xaxis = dict(range=[-180,180]),yaxis = dict(range=[0,-5]) )
#fig3.update_xaxes(nticks=5)
fig2.show()

animation_br_plot_url = py.plot(fig2, filename='animation_br_time', auto_open=False,)
print(animation_br_plot_url)

https://plotly.com/~chigaga/77/


In [70]:
#area in time
area_plot = sum_by_sec.copy()
area_plot['area'] = area_plot['area']/10000000000

sec11 = area_plot.query('sector == 1.0')
sec22 = area_plot.query('sector == 2.0')
sec33 = area_plot.query('sector == 3.0')
sec44 = area_plot.query('sector == 4.0')
sec55 = area_plot.query('sector == 5.0')

fig = make_subplots(rows=2, cols=1,shared_xaxes=True,)

fig.add_trace(go.Scatter(x=sec11['time'], y=sec11['area'],mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='area 1'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec22['time'], y=sec22['area'],mode='lines', line = dict(color='#EF553B'),legendgroup="group2",name='area 2'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec33['time'], y=sec33['area'],mode='lines', line = dict(color='#00CC96'),legendgroup="group3",name='area 3'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec44['time'], y=sec44['area'],mode='lines', line = dict(color='#AB63FA'),legendgroup="group4",name='area 4'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec55['time'], y=sec55['area'],mode='lines', line = dict(color='#FFA15A'),legendgroup="group5",name='area 5'), row=1, col=1)

sm_dy1 = exp_smooth(time_deriv(sec11, 'area'),0.4)
fig.add_trace(go.Scatter(x=sec11['time'], y=sm_dy1,mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='rate 1'), row=2, col=1)
sm_dy2 = exp_smooth(time_deriv(sec22, 'area'),0.4)
fig.add_trace(go.Scatter(x=sec22['time'], y=sm_dy2,mode='lines', line = dict(color='#EF553B'),legendgroup="group2",name='rate 2'), row=2, col=1)
sm_dy3 = exp_smooth(time_deriv(sec33, 'area'),0.4)
fig.add_trace(go.Scatter(x=sec33['time'], y=sm_dy3,mode='lines', line = dict(color='#00CC96'),legendgroup="group3",name='rate 3'), row=2, col=1)
sm_dy4 = exp_smooth(time_deriv(sec44, 'area'),0.4)
fig.add_trace(go.Scatter(x=sec44['time'], y=sm_dy4,mode='lines', line = dict(color='#AB63FA'),legendgroup="group4",name='rate 4'), row=2, col=1)
sm_dy5 = exp_smooth(time_deriv(sec55, 'area'),0.4)
fig.add_trace(go.Scatter(x=sec55['time'], y=sm_dy5,mode='lines', line = dict(color='#FFA15A'),legendgroup="group5",name='rate 5'), row=2, col=1)



#plotting_edge_rate(sec11, mode='area', color='#636EFA',group = 'group1',name='rate 1')
#plotting_edge_rate(sec22, mode='area', color='#EF553B',group = 'group2',name='rate 2')
#plotting_edge_rate(sec33, mode='area', color='#00CC96',group = 'group3',name='rate 3')
#plotting_edge_rate(sec44, mode='area', color='#AB63FA',group = 'group4',name='rate 4')
#plotting_edge_rate(sec55, mode='area', color='#FFA15A',group = 'group5',name='rate 5')

fig.update_layout(title='<b>Area and its growth rate for each sector in time</b>',width=800, height=700)
fig.update_yaxes(title_text="Area [10^10 km^2]", row=1, col=1)
fig.update_yaxes(title_text="Rate [10^7 km^2/s]", row=2, col=1)
fig.update_xaxes(nticks=10,title_text="Time [h]")
fig.show()


area_plot_url = py.plot(fig, filename='area', auto_open=False,)
print(area_plot_url)

https://plotly.com/~chigaga/6/


In [16]:
#@title (old) fun plot derivative
#mode: 'edge' or 'area' - to plot different parameters
def plotting_edge_rate(sec, mode, color,group,name):
  rr = sec
  x = rr['time']
  if mode=='edge':
    y = rr['max']
  else:
    y = rr['area']

  dy = np.zeros(y.shape,np.float)
  dy[0:len(y)-1] = np.diff(y)/np.diff(x)/3600
  dy[-1] = dy[-2]

  return (fig.add_trace(go.Scatter(x=x, y=dy,mode='lines',line = dict(color=color),legendgroup=group,name=name), row=2, col=1))

In [52]:
sm_dy1 = exp_smooth(time_deriv(sec11, 'area'),0.4)

In [55]:
dy1 = time_deriv(sec33, 'area')
fig = go.Figure()
fig.add_trace(go.Scatter(x=sec33['time'], y=dy1, mode='lines', name = 'rate', showlegend = True))
fig.add_trace(go.Scatter(x=sec33['time'], y=sm_dy1, mode='lines',name = 'smoothed rate', showlegend = True))

fig.show()

In [72]:
sector_list = [sec11,sec22,sec33,sec44,sec55]
plot_traces(sector_list, var='area', xlable = 'Time [h]', ylable = 'Area [10^10 km^2]',
                legend = legend_list, title = 'Area evolution for each sector', mode='lines' )

# Leading edge analysis

Figure 0: distribution of pixels by distance

In [ ]:
#@title to plot dist at 1 time step
##if we want to check one time step
t = 1.09
time1 = sectors[sectors['time'] == t]
rr = time1
#sec_string = str(rr['sector'][0])
one_step = rr.groupby(['time','sector','dist']).size().to_frame('count').reset_index()
fig0 = px.line(one_step, x='dist', y='count', color='sector',#line_group="sector",range_x=[-180,180],
                title='<b>Distance of pixels for each sector after 1 hour from the dimming appearance </b>')#, line_close=True)
fig0.update_layout( xaxis_title="Distance from the center [Mm]",  yaxis_title='count')
fig0.update_xaxes(range=[0,500])
fig0.show()

one_time_plot_url = py.plot(fig0, filename='one_time', auto_open=False,)
print(one_time_plot_url)

Figure 1: pixels by distance for all time steps

In [ ]:
#@title all distances distribution 
fig1 = px.line(sum_by_sec_dist, x='dist', y='count', color='sector',line_group="time",# range_x=[-180,180],
            title='<b>Distance of pixels for each sector </b>'  )#, line_close=True)
fig1.update_layout(xaxis_title="Distance from the center [Mm]",   yaxis_title='count')
fig1.update_xaxes(range=[0,400])
fig1.show()


Now we want to create a minimal number of pixels in each ring to be counted as leading edge: we take 10% of each ring's area. 

Figure 2: time profile for the leading edge

In [ ]:
#area of the ring  - we know that radius of each ring is 6 pixels
percent = 0.1 #percent of area enought for the lead edge
sum_by_sec_dist['S'] = (2*sum_by_sec_dist['ring']*6-1)*math.pi*percent

sum_by_sec_dist['edge'] = np.where((sum_by_sec_dist['count'] >= sum_by_sec_dist['S']),sum_by_sec_dist['dist'], np.nan)
sum_by_sec_dist = sum_by_sec_dist.dropna()
distance=sum_by_sec_dist.groupby(['time','sector']).edge.agg([max]).reset_index()
#distance=sum_by_sec_dist.groupby(['time','sector'])['edge'].transform(max).reset_index()
#distance

function to plot the time derivative

In [75]:
#@title Time evolution of distance and its rate
sec1 = distance.query('sector == 1.0')
sec2 = distance.query('sector == 2.0')
sec3 = distance.query('sector == 3.0')
sec4 = distance.query('sector == 4.0')
sec5 = distance.query('sector == 5.0')

fig = make_subplots(rows=2, cols=1,shared_xaxes=True,)
fig.add_trace(go.Scatter(x=sec1['time'], y=sec1['max'],mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='edge 1'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec2['time'], y=sec2['max'],mode='lines', line = dict(color='#EF553B'),legendgroup="group2",name='edge 2'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec3['time'], y=sec3['max'],mode='lines', line = dict(color='#00CC96'),legendgroup="group3",name='edge 3'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec4['time'], y=sec4['max'],mode='lines', line = dict(color='#AB63FA'),legendgroup="group4",name='edge 4'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec5['time'], y=sec5['max'],mode='lines', line = dict(color='#FFA15A'),legendgroup="group5",name='edge 5'), row=1, col=1)

dy1_dist = time_deriv(sec1, 'max')
fig.add_trace(go.Scatter(x=sec1['time'], y=dy1_dist,mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='rate 1'), row=2, col=1)
dy2_dist = time_deriv(sec2, 'max')
fig.add_trace(go.Scatter(x=sec2['time'], y=dy2_dist,mode='lines', line = dict(color='#EF553B'),legendgroup="group2",name='rate 2'), row=2, col=1)
dy3_dist = time_deriv(sec3, 'max')
fig.add_trace(go.Scatter(x=sec3['time'], y=dy3_dist,mode='lines', line = dict(color='#00CC96'),legendgroup="group3",name='rate 3'), row=2, col=1)
dy4_dist = time_deriv(sec4, 'max')
fig.add_trace(go.Scatter(x=sec4['time'], y=dy4_dist,mode='lines', line = dict(color='#AB63FA'),legendgroup="group4",name='rate 4'), row=2, col=1)
dy5_dist = time_deriv(sec5, 'max')
fig.add_trace(go.Scatter(x=sec5['time'], y=dy5_dist,mode='lines', line = dict(color='#FFA15A'),legendgroup="group5",name='rate 5'), row=2, col=1)

fig.update_layout(title='<b>Leading edge and its rate for each sector in time</b>',width=800, height=700)
fig.update_yaxes(title_text="Leading edge [Mm]", row=1, col=1)
fig.update_yaxes(title_text="Rate [Mm/s]", row=2, col=1)
fig.update_xaxes(nticks=10,title_text="Time [h]")
fig.show()

lead_edge_plot_url = py.plot(fig, filename='lead_edge', auto_open=False,)
print(lead_edge_plot_url)

https://plotly.com/~chigaga/75/


## Time evolution of area and *brightness*